In [4]:
# 导入配置加载器
from config_loader import load_config

# ==================== 加载配置 ====================
print("=" * 60)
print("加载配置文件")
print("=" * 60)

try:
    config = load_config()
    zhipuai_config = config.get_model_config('zhipuai')
    print(f"\n✅ 配置加载成功！")
    print(f"  模型: {zhipuai_config['model']}")
    print(f"  温度: {zhipuai_config['temperature']}")
except Exception as e:
    print(f"\n❌ 配置加载失败: {e}")
    print("\n请按照以下步骤配置:")
    print("1. 复制 .env.example 为 .env")
    print("   命令: cp .env.example .env")
    print("2. 编辑 .env 文件，填写 ZHIPUAI_API_KEY")
    print("3. 保存后重新运行此单元格")
    raise


加载配置文件
✅ 已加载配置文件: /Users/yuyansong/AiProject/Langchain/langchain-example/langchain/.env

✅ 配置加载成功！
  模型: glm-4.5-air
  温度: 0.7


In [6]:
from langchain.agents import create_agent
from langchain_community.chat_models import ChatZhipuAI

model = ChatZhipuAI(**zhipuai_config)

# ✅ v1.0：直接使用字符串或字典
# 方式1：纯字符串（简单场景）
response = model.invoke("翻译成英文：你好世界")
print("方式1：")
print(response)

# 方式2：消息列表（多轮对话）
messages = [
    {"role": "system", "content": "你是一个专业的翻译助手"},
    {"role": "user", "content": "翻译：Hello World"}
]
response = model.invoke(messages)
print("方式2：")
print(response)

# 方式3：使用 Message 对象（兼容旧代码）
from langchain_core.messages import SystemMessage, HumanMessage
messages = [
    SystemMessage(content="你是一个专业的翻译助手"),
    HumanMessage(content="翻译：Hello World")
]
response = model.invoke(messages)
print("方式3：")
print(response)

方式1：
content='Hello World!  \n\n（注：根据使用场景，也可翻译为 "Hello, world!" 带逗号更符合日常英语习惯。在编程领域固定写法为 "Hello World" 不加标点。）' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 253, 'prompt_tokens': 11, 'prompt_tokens_details': {'cached_tokens': 4}, 'total_tokens': 264}, 'model_name': 'glm-4.5-air', 'finish_reason': 'stop'} id='lc_run--019af92f-6020-78e0-909e-672f72781f9a-0'
方式2：
content='你好，世界' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 16, 'prompt_tokens_details': {'cached_tokens': 6}, 'total_tokens': 74}, 'model_name': 'glm-4.5-air', 'finish_reason': 'stop'} id='lc_run--019af92f-6b54-7f93-8f1f-3c08ffb121a9-0'
方式3：
content='你好，世界' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 16, 'prompt_tokens_details': {'cached_tokens': 15}, 'total_tokens': 71}, 'model_name': 'glm-4.5-air', 'finish_reason': 'stop'} id='lc_run--019af92f-78d2-7021-b06d-d795964ff580-0'


In [7]:
system_prompt = """你是一个专业的 Python 编程助手。

你的特点：
- 擅长解释复杂概念
- 代码示例清晰易懂
- 注重最佳实践
- 不使用已废弃的语法

回答要求：
- 先解释原理，再给代码
- 代码要有详细注释
- 如果有多种方案，说明优劣
"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "什么是装饰器？"}
]

response = model.invoke(messages)
print(response.content)

# Python装饰器详解

装饰器(Decorator)是Python中一种强大的编程工具，它允许我们修改或增强函数和方法的行为，而无需永久性地修改它们。装饰器是Python语言的核心特性之一，广泛应用于日志记录、性能测试、权限验证等场景。

## 装饰器的原理

装饰器的核心原理基于Python的几个重要特性：

1. **一切皆对象**：在Python中，函数是"一等公民"，可以作为参数传递给其他函数，也可以作为返回值
2. **嵌套函数**：可以在一个函数内部定义另一个函数
3. **闭包**：内部函数可以访问外部函数的变量

装饰器本质上是一个函数，它接受一个函数作为参数，并返回一个新的函数。这个新函数通常会先执行一些额外操作，然后调用原始函数，最后可能还会执行一些额外操作。

## 简单的装饰器示例

```python
# 定义一个装饰器函数
def my_decorator(func):
    def wrapper():
        print("函数执行前的操作")
        func()  # 调用原始函数
        print("函数执行后的操作")
    return wrapper

# 使用装饰器
@my_decorator
def say_hello():
    print("Hello!")

# 调用say_hello实际上是调用wrapper函数
say_hello()
```

输出：
```
函数执行前的操作
Hello!
函数执行后的操作
```

这等价于：
```python
def say_hello():
    print("Hello!")

# 手动应用装饰器
say_hello = my_decorator(say_hello)
```

## 带参数的装饰器

当被装饰的函数需要接受参数时，我们需要修改装饰器：

```python
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("函数执行前的操作")
        result = func(*args, **kwargs)  # 调用原始函数并获取结果
        print("函数执行后的操作")
        return

In [8]:
"""
Few-shot 示例：情感分类
"""

few_shot_prompt = """你是一个情感分析助手。根据文本判断情感倾向。

示例1：
输入：今天天气真好，心情很愉快！
输出：正面 | 置信度：0.95

示例2：
输入：产品质量太差了，非常失望。
输出：负面 | 置信度：0.90

示例3：
输入：今天去了超市买东西。
输出：中性 | 置信度：0.85

现在分析下面的文本：
"""

user_input = "这个课程内容很实用，学到了很多！"

messages = [
    {"role": "system", "content": few_shot_prompt},
    {"role": "user", "content": user_input}
]

response = model.invoke(messages)
print(response.content)
# 期望输出：正面 | 置信度：0.95

正面 | 置信度：0.95


In [9]:
"""
提示词模板示例
"""

class PromptTemplate:
    """简单的提示词模板类"""

    def __init__(self, template: str):
        self.template = template

    def format(self, **kwargs) -> str:
        return self.template.format(**kwargs)

# 定义模板
translation_template = PromptTemplate(
    """你是一个专业的翻译助手。

任务：将 {source_lang} 翻译成 {target_lang}
风格：{style}

原文：
{text}

翻译："""
)

# 使用模板
prompt = translation_template.format(
    source_lang="中文",
    target_lang="英文",
    style="正式",
    text="人工智能正在改变世界"
)

response = model.invoke(prompt)
print(response.content)

Artificial Intelligence is changing the world.


# 批量调用与异步处理

In [10]:
# 准备多个输入
inputs = [
    "翻译成英文：春天来了",
    "翻译成英文：夏天很热",
    "翻译成英文：秋天落叶",
    "翻译成英文：冬天下雪"
]

# ✅ 批量调用（高效）
import time

start = time.time()
# 批量调用与异步处理
responses = model.batch(inputs)
batch_time = time.time() - start

print("批量调用结果：")
for i, response in enumerate(responses):
    print(f"{i+1}. {response.content}")
print(f"耗时: {batch_time:.2f}秒\n")

# ❌ 循环调用（低效，仅用于对比）
start = time.time()
loop_responses = []
for inp in inputs:
    response = model.invoke(inp)
    loop_responses.append(response)
loop_time = time.time() - start

print(f"循环调用耗时: {loop_time:.2f}秒")
print(f"批量调用节省: {((loop_time - batch_time) / loop_time * 100):.1f}%")
     

批量调用结果：
1. Spring is here.
2. Summer is very hot.
3. Autumn leaves
4. It snows in winter.
耗时: 7.74秒

循环调用耗时: 41.02秒
批量调用节省: 81.1%


In [11]:
import asyncio
async def translate_async(text: str) -> str:
    """异步翻译"""
    response = await model.ainvoke(f"翻译成英文：{text}")
    return response.content

async def main():
    # 并发处理多个任务
    tasks = [
        translate_async("春天来了"),
        translate_async("夏天很热"),
        translate_async("秋天落叶"),
        translate_async("冬天下雪")
    ]

    results = await asyncio.gather(*tasks)

    for i, result in enumerate(results, 1):
        print(f"{i}. {result}")

# 运行异步代码
# asyncio.run(main()) # Original line that caused the error
await main()

1. Spring has sprung.
2. Summer is very hot.
3. Autumn leaves
4. It snows in winter.


# 使用 Pydantic 定义输出结构

In [12]:
from pydantic import BaseModel, Field
# 定义输出结构
class SentimentAnalysis(BaseModel):
    """情感分析结果"""
    sentiment: str = Field(description="情感倾向：positive/negative/neutral")
    confidence: float = Field(description="置信度，0-1之间")
    keywords: list[str] = Field(description="关键词列表")


# ✅ v1.0：使用 with_structured_output
structured_model = model.with_structured_output(SentimentAnalysis)

# 调用
text = "这个课程内容很实用，学到了很多知识，强烈推荐！"
result = structured_model.invoke(
    f"分析以下文本的情感：\n{text}"
)

print(f"类型: {type(result)}")  # 
print(f"情感: {result.sentiment}")
print(f"置信度: {result.confidence}")
print(f"关键词: {result.keywords}")

类型: <class '__main__.SentimentAnalysis'>
情感: positive
置信度: 0.95
关键词: ['实用', '学到知识', '强烈推荐']


# 复杂结构输出

In [13]:
from pydantic import BaseModel
from typing import List

class Aspect(BaseModel):
    """评论维度"""
    name: str = Field(description="维度名称，如：质量、价格、服务")
    score: int = Field(description="评分，1-5")
    comment: str = Field(description="具体评价")

class ProductReview(BaseModel):
    """产品评论分析"""
    overall_sentiment: str = Field(description="整体情感：positive/negative/neutral")
    overall_score: int = Field(description="综合评分，1-5")
    aspects: List[Aspect] = Field(description="各维度评价")
    summary: str = Field(description="一句话总结")

# 创建结构化模型
structured_model = model.with_structured_output(ProductReview)
# 测试
review_text = """
这款笔记本电脑性能非常强大，运行大型软件毫无压力。
屏幕色彩鲜艳，看视频很舒服。
不过价格有点贵，而且风扇噪音较大。
客服态度很好，物流也快。
总体来说还是值得购买的。
"""

result = structured_model.invoke(
    f"分析以下产品评论：\n{review_text}"
)

print(f"整体情感: {result.overall_sentiment}")
print(f"综合评分: {result.overall_score}/5")
print(f"\n各维度评价:")
for aspect in result.aspects:
    print(f"  - {aspect.name}: {aspect.score}/5 - {aspect.comment}")
print(f"\n总结: {result.summary}")

整体情感: positive
综合评分: 4/5

各维度评价:
  - 性能: 5/5 - 性能非常强大，运行大型软件毫无压力
  - 屏幕: 4/5 - 屏幕色彩鲜艳，看视频很舒服
  - 价格: 2/5 - 价格有点贵
  - 噪音: 2/5 - 风扇噪音较大
  - 服务: 5/5 - 客服态度很好，物流也快

总结: 性能出色但价格偏高的笔记本电脑


In [14]:
"""
实战项目：智能文案生成器（LangChain 1.0）

功能：
- 多种文案类型和风格
- 批量生成
- AI 自动评分
- 导出到文件
"""

from pydantic import BaseModel, Field
from typing import List, Literal
import json
from datetime import datetime

# 定义结构化输出
class CopywritingScore(BaseModel):
    """文案评分"""
    creativity: int = Field(description="创意性，1-10分")
    clarity: int = Field(description="清晰度，1-10分")
    appeal: int = Field(description="吸引力，1-10分")
    overall: int = Field(description="综合评分，1-10分")
    comment: str = Field(description="简短评语")

class Copywriting(BaseModel):
    """文案结果"""
    content: str = Field(description="文案内容")
    score: CopywritingScore = Field(description="质量评分")

class CopywritingBatch(BaseModel):
    """批量文案结果"""
    copies: List[Copywriting] = Field(description="文案列表")

class SmartCopywriter:
    """智能文案生成器"""

    def __init__(self):
        # 初始化模型
        self.model = ChatZhipuAI(**zhipuai_config)

        # 结构化模型
        self.structured_model = self.model.with_structured_output(CopywritingBatch)

    def generate(
        self,
        product: str,
        copy_type: Literal["广告", "社交媒体", "邮件", "博客"],
        style: Literal["正式", "轻松", "幽默", "专业"],
        count: int = 3
    ) -> List[Copywriting]:
        """
        生成文案

        Args:
            product: 产品或服务描述
            copy_type: 文案类型
            style: 文案风格
            count: 生成数量

        Returns:
            文案列表（带评分）
        """

        prompt = f"""你是一个专业的文案撰写专家。

          任务：为以下产品创作{copy_type}文案

          产品信息：
          {product}

          要求：
          - 文案类型：{copy_type}
          - 文案风格：{style}
          - 生成数量：{count}个不同版本
          - 每个文案都要有创意，不要重复
          - 为每个文案打分并给出评语

          输出：
          请输出{count}个文案，每个文案包含内容和评分。
          """

        result = self.structured_model.invoke(prompt)
        return result.copies

    def export_to_file(self, copies: List[Copywriting], filename: str = None):
        """导出到文件"""
        if filename is None:
            filename = f"copywriting_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

        data = {
            "generated_at": datetime.now().isoformat(),
            "copies": [
                {
                    "content": c.content,
                    "score": {
                        "creativity": c.score.creativity,
                        "clarity": c.score.clarity,
                        "appeal": c.score.appeal,
                        "overall": c.score.overall,
                        "comment": c.score.comment
                    }
                }
                for c in copies
            ]
        }

        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        return filename

def main():
    """主函数"""
    copywriter = SmartCopywriter()

    print("=" * 70)
    print("🚀 智能文案生成器（LangChain 1.0）")
    print("=" * 70)

    # 示例1：广告文案
    print("\n【示例1】生成广告文案")
    print("-" * 70)

    product1 = """
    产品：AI 编程助手
    特点：
    - 实时代码补全
    - 智能错误检测
    - 支持 20+ 编程语言
    - 提高开发效率 50%
    """

    copies1 = copywriter.generate(
        product=product1,
        copy_type="广告",
        style="专业",
        count=3
    )

    for i, copy in enumerate(copies1, 1):
        print(f"\n版本 {i}:")
        print(f"内容: {copy.content}")
        print(f"评分: 创意{copy.score.creativity} | 清晰度{copy.score.clarity} | "
              f"吸引力{copy.score.appeal} | 综合{copy.score.overall}")
        print(f"评语: {copy.score.comment}")

    # 示例2：社交媒体文案
    print("\n" + "=" * 70)
    print("【示例2】生成社交媒体文案")
    print("-" * 70)

    product2 = """
      活动：LangChain 1.0 完整课程上线
      内容：
      - 10 节完整课程
      - 基于最新 1.0 版本
      - 包含实战项目
      - 适合 Python 开发者
      """

    copies2 = copywriter.generate(
        product=product2,
        copy_type="社交媒体",
        style="轻松",
        count=3
    )

    for i, copy in enumerate(copies2, 1):
        print(f"\n版本 {i}:")
        print(f"内容: {copy.content}")
        print(f"综合评分: {copy.score.overall}/10")

    # 导出
    print("\n" + "=" * 70)
    filename = copywriter.export_to_file(copies1 + copies2)
    print(f"✅ 已导出到文件: {filename}")

    # 交互模式
    print("\n" + "=" * 70)
    print("💬 进入交互模式（输入 'quit' 退出）")
    print("=" * 70)

    while True:
        print("\n请输入产品信息（可多行，输入空行结束）:")
        lines = []
        while True:
            line = input()
            if line == "":
                break
            if line.lower() == "quit":
                return
            lines.append(line)

        if not lines:
            continue

        product = "\n".join(lines)

        copy_type = input("文案类型（广告/社交媒体/邮件/博客）: ") or "广告"
        style = input("文案风格（正式/轻松/幽默/专业）: ") or "专业"
        count = int(input("生成数量（默认3）: ") or "3")

        print("\n🤖 正在生成文案...\n")

        copies = copywriter.generate(
            product=product,
            copy_type=copy_type,
            style=style,
            count=count
        )

        for i, copy in enumerate(copies, 1):
            print(f"\n{'='*60}")
            print(f"版本 {i}")
            print(f"{'='*60}")
            print(copy.content)
            print(f"\n📊 评分: {copy.score.overall}/10 - {copy.score.comment}")

        save = input("\n是否保存？(y/n): ")
        if save.lower() == 'y':
            filename = copywriter.export_to_file(copies)
            print(f"✅ 已保存到: {filename}")

if __name__ == "__main__":
    main()

🚀 智能文案生成器（LangChain 1.0）

【示例1】生成广告文案
----------------------------------------------------------------------

版本 1:
内容: AI编程助手——让代码编写如行云流水。智能补全每一行，实时检测每一个错误，支持20+编程语言，开发效率提升50%。不再是繁琐的调试，而是专注创新的专业体验。
评分: 创意8 | 清晰度9 | 吸引力8 | 综合8
评语: 文案流畅自然，突出了产品的核心优势

版本 2:
内容: 20+编程语言，一个AI助手搞定。实时补全智能纠错，开发效率提升50%。从JavaScript到Python，从Java到C++，AI编程助手让编码更智能，让开发更高效。
评分: 创意7 | 清晰度9 | 吸引力8 | 综合8
评语: 清晰列举了多语言支持的优势，专业性强

版本 3:
内容: 告别编码焦虑，拥抱AI时代。AI编程助手——您的24小时智能编程伙伴。实时代码补全，智能错误预警，多语言无缝切换，效率提升50%。让每一行代码都充满智慧。
评分: 创意9 | 清晰度8 | 吸引力9 | 综合9
评语: 创意性强，情感共鸣足，突出了AI助手的陪伴感

【示例2】生成社交媒体文案
----------------------------------------------------------------------

版本 1:
内容: 🚀 LangChain 1.0 完整课程上线！10节干货，带你从零到AI大师！✨ 最新1.0版本，实战项目练手，Python开发者专属福利！还在等什么？立即加入，让AI为你打工！💪 #LangChain #AI开发 #Python
综合评分: 9/10

版本 2:
内容: 🎯 Python开发者看过来！LangChain 1.0课程终于来了！10节完整教程，1.0最新版本，实战项目全解锁！告别理论，直接上手！学完就能用，让你的技能up up！🔥 #编程学习 #AI技术 #实战项目
综合评分: 8/10

版本 3:
内容: 🌟 AI时代必备技能！LangChain 1.0完整课程震撼上线！📚 10节精品课，最新1.0版本，实战项目练不停！Python开发者的进阶秘籍，让你在AI浪潮中乘风破浪！🎓 #AI学习 #技